# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

/Users/lamiakasmi/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lamiakasmi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lamiakasmi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lamiakasmi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.grid_search import GridSearchCV

/Users/lamiakasmi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/lamiakasmi/anaconda3/lib/python3.7/site-packages/sklearn/grid_search.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, namedtuple, Sized
/Users/lamiakasmi/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", Depre

In [3]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql_table('Messages',engine)
    X = df['message']
    Y = df.iloc[:,4:].drop(columns='child_alone')
    return X,Y
X,Y = load_data()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def build_pipeline():
    model = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('clf', MultiOutputClassifier(RandomForestClassifier()))
                        ])
    return model

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
def train(X,Y,model):
    X_train, X_test, y_train, y_test = train_test_split(X,Y) 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = pd.DataFrame(y_pred, index = y_test.index, columns = y_test.columns)
    return X_train, X_test, y_train, y_test, y_pred
model = build_pipeline()
X_train, X_test, y_train, y_test, y_pred = train(X,Y,model)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def display_results(y_test, y_pred):
    for i in range(y_test.shape[1]):
        print(70*'='+'\nFeature: ',Y.columns[i])
        print(classification_report(y_test.iloc[:,i],y_pred.iloc[:,i]))
display_results(y_test,y_pred)

Feature:  related
             precision    recall  f1-score   support

          0       0.61      0.48      0.54      1537
          1       0.85      0.90      0.87      4965
          2       0.30      0.29      0.29        52

avg / total       0.79      0.80      0.79      6554

Feature:  request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5444
          1       0.77      0.46      0.57      1110

avg / total       0.88      0.88      0.87      6554

Feature:  offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

Feature:  aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.80      3891
          1       0.73      0.60      0.66      2663

avg / total       0.75      0.75      0.74      6554

Feature:  med

/Users/lamiakasmi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [6]:
def build_pipeline():
    pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
                        ])
    return pipeline

In [8]:
def build_model():
    pipeline = build_pipeline()
    parameters = {
                'vect__max_features': (None, 5000, 10000),
                'clf__estimator__n_estimators': [50, 100],
                'clf__estimator__min_samples_split': [2, 3]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    return cv

In [9]:
def train_model(X,Y,model):
    X_train, X_test, y_train, y_test = train_test_split(X,Y) 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = pd.DataFrame(y_pred, index = y_test.index, columns = y_test.columns)    
    return X_train, X_test, y_train, y_test, y_pred

model = build_pipeline()
X_train, X_test, y_train, y_test, y_pred = train_model(X,Y,model)

In [14]:
display_results(y_test,y_pred)

Feature:  related
             precision    recall  f1-score   support

          0       0.79      0.20      0.32      1539
          1       0.79      0.98      0.88      4971
          2       0.30      0.14      0.19        44

avg / total       0.79      0.79      0.74      6554

Feature:  request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5445
          1       0.83      0.49      0.62      1109

avg / total       0.89      0.90      0.89      6554

Feature:  offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.04      0.03      0.03        31

avg / total       0.99      0.99      0.99      6554

Feature:  aid_related
             precision    recall  f1-score   support

          0       0.74      0.89      0.81      3827
          1       0.78      0.56      0.65      2727

avg / total       0.75      0.75      0.74      6554

Feature:  med

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
display_results(y_test,y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
def display_results(y_test, y_pred):
    for i in range(y_test.shape[1]):
        print(70*'='+'\nFeature: ',Y.columns[i])
        print(classification_report(y_test.iloc[:,i],y_pred.iloc[:,i]))

In [37]:
display_results(y_test,y_pred)

Feature:  related
             precision    recall  f1-score   support

          0       0.62      0.35      0.45      1504
          1       0.83      0.93      0.88      5013
          2       0.48      0.38      0.42        37

avg / total       0.78      0.80      0.78      6554

Feature:  request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5455
          1       0.83      0.39      0.53      1099

avg / total       0.88      0.88      0.87      6554

Feature:  offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

Feature:  aid_related
             precision    recall  f1-score   support

          0       0.73      0.87      0.80      3839
          1       0.75      0.55      0.64      2715

avg / total       0.74      0.74      0.73      6554

Feature:  med

/Users/lamiakasmi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.